# Extract

In [ ]:
import gzip             #importo la librería para leer los archivos gzip, formato de compresion en el que vinieron los archivos
import pandas as pd     #importo librería pandas que vamos a usar para el data frame
import ast              #importo librería ast que me ayuda a leer la gramatica abstracta

### Steamgames

In [ ]:
#si bien se podría descomprimir el archivo y directamente leerlo de pandas, dado que futuras actualziaciones pueden venir como gzip, es preferible trabajarlos desde ese formato
with gzip.open('Datos_sin_procesar/steam_games.json.gz', 'rb') as f:
    df_steamgames=pd.read_json(f, lines=True)

### User Items

In [ ]:
#comienzo con una lista para poder insertar cada línea del archivo
data=[]
with gzip.open('Datos_sin_procesar/users_items.json.gz', 'rb') as f:
    for line in f:
        #cargar la información usando ast, porque si se usa json se dan errores por sintacis de gramatica en el uso de comillas
        data.append(ast.literal_eval(line.decode('utf-8')))

# Convierto la lista a DataFrame
df_users_items = pd.DataFrame(data)

### User Reviews

In [ ]:
#comienzo con una lista para poder insertar cada línea del archivo
data=[]
with gzip.open('Datos_sin_procesar/user_reviews.json.gz', 'rb') as f:
    for line in f:
        #cargar la información usando ast, porque si se usa json se dan errores por sintacis de gramatica en el uso de comillas
        data.append(ast.literal_eval(line.decode('utf-8')))

# Convierto la lista a DataFrame
df_user_reviews = pd.DataFrame(data)

# Transorm

### Steamgames

In [ ]:
#se eliminan los valores con todos nulos
df_steamgames.dropna(inplace= True, how='all')

In [ ]:
#verifico algún id que esté nulo.
df_steamgames[df_steamgames['id'].isnull()]

In [ ]:
#se verifica sin con la url podemos obetener el id del juego
df_steamgames[['url','id']]
#se concluye que sí, y se pasa a asignar ese valor a los faltantes

In [ ]:
#se importa la libreria re para poder trabajar sobre la extracción del ID de la url, y se genera una función para este propósito
import re

def extraer_numero_url(url):
    if url is None:
        return None
    else:
        match = re.search(r'(\d+)', url)
        if match:
            return match.group(1)
        else:
            return None

#aplicamos la funcion a una columna auxiliar
df_steamgames['id_aux']=df_steamgames['url'].apply(extraer_numero_url)
#aplicamos la columna auxiliar a los valores nulos
df_steamgames['id'][df_steamgames['id'].isnull()]=df_steamgames['id_aux']
#eliminamos los id nulos
df_steamgames.dropna(subset=['id'], inplace=True)
#eliminamos la columna auxiliar
df_steamgames.drop(columns=['id_aux'], inplace=True)


La columna genres tiene listas con los generos a los que puede pertenecer el juego. Para un mejor procesamiento a posteriori de los datos, se transforma esa columna en varias columnas dummies con cada uno de esos valores.
También se tiene la columna tags, pero que tiene muchos más valores. Algunos de ellos son las categorías, y también otros tipos de etiquetas. Se arma un dataframe para rellenar los valores faltantes de etiquetas.

In [ ]:
#Lleno el title nulo con el app_name
df_steamgames.loc[df_steamgames['title'].isnull(), 'title'] = df_steamgames['app_name']
#cambio los nombres de columas que se usan más adelante
df_steamgames = df_steamgames.rename(columns={'id': 'item_id'})
df_steamgames = df_steamgames.rename(columns={'title': 'item_title'})

In [ ]:
#Lleno la columna tags nula con la columna genres. Esto se utiliza luego en el algotirmo de ML
df_steamgames.loc[df_steamgames['tags'].isnull(), 'tags'] = df_steamgames['genres']

In [ ]:
#primero llenamos los nulos con valor [], para su posterior procesamiento

# Define una función para reemplazar los valores nulos con una lista vacía
def reemplazonulo(lst):
    return lst if isinstance(lst, list) else []

# Aplica la función a la columna con listas
df_steamgames['genres'] = df_steamgames['genres'].apply(reemplazonulo)
df_steamgames['tags'] = df_steamgames['tags'].apply(reemplazonulo)

In [ ]:
#Separo la columna de generos
df_genres = df_steamgames['genres'].apply(lambda x: pd.Series({val: 1 for val in x}))
#lleno los nulos con 0
df_genres.fillna(0, inplace=True)
#Se renombran columnas que tienen mal los nombres
df_genres.rename(columns={'Design &amp; Illustration':'Design & Illustration','Animation &amp; Modeling':'Animation & Modeling'}, inplace= True)
#armo una lista con los generos
list_Genres=df_genres.columns.to_list()


In [ ]:
list_Genres

In [ ]:
#Se genera el dataframe de de tags
df_tags = df_steamgames['tags'].apply(lambda x: pd.Series({val: 1 for val in x}))
df_tags.fillna(0, inplace=True)
#idem caso anterior con nombres mal importados
df_tags.rename(columns={'Design &amp; Illustration':'Design & Illustration','Animation &amp; Modeling':'Animation & Modeling'}, inplace= True)

In [ ]:
#generamos un dataframe que usaremos luego en el algoritmo de ML
df_tags_for_ML=pd.concat([df_steamgames[['item_title', 'item_id']],df_tags], axis=1)
#cambiamos de formato sus columnas para que ocupen menos espacio
columnas=df_tags_for_ML.columns.tolist()
columnas.remove('item_title')
for i in columnas:
    df_tags_for_ML[i]= df_tags_for_ML[i].astype(int)
#se exporta el dataframe, y se elimina
df_tags_for_ML.to_csv('Datos_procesados/df_tags_for_ML.csv')
#se elimina para que no ocupe más espacio
del df_tags_for_ML

In [ ]:
'''
verificamos que todos los géneros están en la lista
tags=df_tags.columns.tolist()
genres=df_genres.columns.tolist()
genres_not_in_tag=[]
for i in genres:
    if i not in tags: print(i+' no está')
    else:
        print(i+' está')
'''

In [ ]:
#sacamos el genero Accounting que no se encuentra en ningún tag
#genres.remove('Accounting')
genres=df_genres.columns.tolist()
#completamos las categorías que son nulas en la columna de categoria pero si fueron etiquetadas usando los tags
for i in genres:
    for index, row in df_genres.iterrows():
        if row[i] == 0:
            df_genres.at[index, i] = df_tags.at[index, i]
df_steamgames=pd.concat([df_steamgames,df_genres], axis=1)

#eliminamos los dataframes que ya no usamos
del df_genres
del df_tags

#volvemos a agregar el genero Accounting
genres.append('Accounting')

In [ ]:
#Pongo el año de lanzamiento
df_steamgames['release_date'] = pd.to_datetime(df_steamgames['release_date'], errors='coerce')
df_steamgames.loc[~df_steamgames['release_date'].notnull(), 'release_date'] = None
df_steamgames['release_year'] = df_steamgames['release_date'].dt.year

In [ ]:
#cambio los tipos de datos para ocupar menos espacio
df_steamgames['release_year'] = df_steamgames['release_year'].fillna(0)
df_steamgames['release_year'] = df_steamgames['release_year'].astype(int)
for i in genres:
    df_steamgames[i] = df_steamgames[i].fillna(0)
    df_steamgames[i] = df_steamgames[i].astype(int)
df_steamgames['item_id']=df_steamgames['item_id'].astype(int)

In [ ]:
#elimino lo que no sirve
df_steamgames.drop(['publisher','app_name','genres', 'tags' ,'url','release_date','reviews_url','specs','price','early_access',
                                 'developer'],axis=1,inplace=True)

### User Items

Se posee en la columna items listas con diccionarios. Cada diccionario tiene 4 atributos. Se genera una fila para cada diccionario, y 4 columnas posteriores a partir del diccionario

In [ ]:
#primero se inserta una fila por cada diccionario de la columna item
df_users_items=df_users_items.explode('items')
df_users_items = df_users_items.reset_index(drop=True)

In [ ]:
#luego se arma un dataframe nuevo a partir de los diccionarios de item, con una columna por cada atributo
# el metodo normal sería este, pero tarda mucho, con lo cual buscamos extraer columna por columna df_users_items_items=df_users_items['items'].apply(pd.Series)

def obtener_elemento(diccionario, clave_busqueda):
    if isinstance(diccionario, dict):
        return diccionario.get(clave_busqueda)
    else:
        return diccionario

df_users_items['item_id'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'item_id'))
df_users_items['item_name'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'item_name'))
df_users_items['playtime_forever'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'playtime_forever'))
df_users_items['playtime_2weeks'] = df_users_items['items'].apply(lambda x: obtener_elemento(x, 'playtime_2weeks'))

In [ ]:
#Saco las columnas items que ya no no utiles
df_users_items.drop(['steam_id','user_url','items', 'items_count'], axis=1,inplace= True)
#eliminamos las filas que no tienen valor de juego
df_users_items = df_users_items.dropna(subset=['item_id', 'item_name'], how='all')

### User Reviews

Cada objeto de la columna review es una lista que tiene varios valores de diccionario, de 7 atributos 'funny', 'posted',   'last_edited', 'item_id', 'helpful', 'recommend', 'review'. Se debe transformar cada diccionario en una fila, y cada uno de estos 7 atriburos en una columna, de manera tal que el dataframe refleje la realidad de datos que tenemos

In [ ]:
#paso cada lista a una fila, aplicando metodo explode
df_user_reviews=df_user_reviews.explode('reviews')
#paso cada atributo del diccionario a una columna nueva, aplicando metodo apply
df_user_reviews_exp=df_user_reviews['reviews'].apply(pd.Series)
df_user_reviews.reset_index()

In [ ]:
#junto el dataframe original y las nuevas columnas
df_user_reviews=pd.concat([df_user_reviews, df_user_reviews_exp], axis=1)
#elimino la columna original
df_user_reviews.drop(['reviews','user_url', 'funny', 'last_edited', 
                      'helpful', 0], axis=1, inplace=True)

In [ ]:
from textblob import TextBlob
df_user_reviews['review'] = df_user_reviews['review'].astype(str)
df_user_reviews['polarity'] = df_user_reviews['review'].apply(lambda text: TextBlob(text).sentiment.polarity)


In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_user_reviews['polarity'], bins=100)

In [ ]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['polarity'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))
df_user_reviews.drop(['review','polarity'], axis=1, inplace=True)

In [ ]:
#sacar el año de la review
df_user_reviews['date'] = df_user_reviews['posted'].str.extract(r'(\w+\s\d+(?:,\s\d{4})?)')
df_user_reviews['date'] = pd.to_datetime(df_user_reviews['date'], format='%B %d, %Y', errors='coerce')
df_user_reviews['year'] = df_user_reviews['date'].dt.year
# Convertir los NaN a 0 en la columna 'year'
df_user_reviews['year'] = df_user_reviews['year'].fillna(0)
# Convertir la columna 'year' a tipo entero
df_user_reviews['year'] = df_user_reviews['year'].astype(int)

In [ ]:
#dado que el ultimo año de las reseñas que tienen fecha es el 2015, parece que fueron realizadas en el año 2016,
# y cuando se realizaban en ese año no se ponía la fecha. Por eso relleno el valor de los años de reseña faltante 
# con el mayor entre ese año, y el año en que se lanzó el juego
df_steamgames.reset_index()

# Definir una función personalizada para aplicar la lógica deseada
def fill_year(row):
    if row['year'] == 0:
        # Obtener las filas de df_steamgames que cumplen con la condición
        steamgame_rows = df_steamgames[df_steamgames['item_id'] == row['item_id']]
        
        if not steamgame_rows.empty:
            # Si se encontraron filas, obtener el año de la primera fila y verificar si es mayor a 2016
            release_year = steamgame_rows['release_year'].iloc[0]
            return max(release_year, 2016)
        else:
            # Si no se encontraron filas, retornar 2016
            return 2016
    else:
        return row['year']

# Aplicar la función personalizada a la columna 'year' utilizando apply
df_user_reviews['year'] = df_user_reviews.apply(fill_year, axis=1)

# Convertir la columna 'year' a tipo entero
df_user_reviews['year'] = df_user_reviews['year'].astype(int)

In [ ]:
#Agrupo lo que me sirve
df_user_item_played_year = df_user_reviews.groupby(['item_id', 'user_id', 'year']).size().reset_index(name='count')
df_user_item_played_year.drop('count', axis=1, inplace=True)
df_user_item_played_year['item_id']=df_user_item_played_year['item_id'].astype(int)

# Transform combinations

#### df_genres_detail

In [ ]:
#Agrego las horas totales de cada juego
#primero armo un dataframe que es la suma total de horas jugadas de cada juego
df_hoursxgame = df_users_items.groupby(by=['item_id', 'item_name'])['playtime_forever'].sum().reset_index()
#cambio el tipo de item_id a integer
df_hoursxgame['item_id']=df_hoursxgame['item_id'].astype(int)
#agrego las horas al dataframe de steamgames
df_steamgames = df_steamgames.merge(df_hoursxgame[['item_id', 'playtime_forever']], on='item_id', how='left')

In [ ]:
#genero una lista, que luego paso a dataframe, en la que busco el año con el maximo de horas jugadas
genres_RY_list=[]
#voy iterando por cada género
for i in genres:
    # genero un dataframe con cada año de lanzamiento y las horas jugadas, un agrupado del dataframe steamgames
    df_append = df_steamgames[df_steamgames[i] == 1].groupby(by=['release_year'])['playtime_forever'].sum().reset_index()
    # Encuentro el año con la máxima cantidad de horas jugadas
    max_hours_year = df_append.loc[df_append['playtime_forever'].idxmax(), 'release_year']
    # Agrego los datos a la lista de géneros y años de máximas horas jugadas
    genres_RY_list.append({'Genre': i, 'RYMaxHours': max_hours_year})
df_genres=pd.DataFrame(genres_RY_list)
df_genres['Genre']=df_genres['Genre'].astype(str)

In [ ]:
#cambio el tipo de valor de item id
df_users_items['item_id']=df_users_items['item_id'].astype(int)
#Incluyo los generos en el dataframe de los usuarios
df_users_items = df_users_items.merge(df_steamgames, on='item_id', how='left')
#cambio nomnbre y elimino columnas duplicadas
df_users_items.rename(columns={'playtime_forever_x':'playtime_forever'}, inplace=True)
df_users_items.drop('playtime_forever_y', axis=1, inplace=True)
#convierto a enteros las columnas de generos para que ocupe menos memoria
for i in genres:
    df_users_items[i] = df_users_items[i].fillna(0).astype(int)

In [ ]:
## JUGADOR QUE MAS HORAS JUGO

#genero una lista, que luego paso a dataframe, en la que busco el jugador con el maximo de horas jugadas
genres_MaxPlayer_list=[]
#voy iterando por cada género
for index, row in df_genres.iterrows():
    genero = row['Genre']
    # genero un dataframe con cada jugador y las horas jugadas de ese genero, un agrupado del dataframe df_users_items
    df_append = df_users_items[df_users_items[genero] == 1].groupby(by=['user_id'])['playtime_forever'].sum().reset_index()
    if not df_append.empty:
        # Encontrar el jugador con la máxima cantidad de horas jugadas
        max_hours_player = df_append.loc[df_append['playtime_forever'].idxmax(), 'user_id']
    else:
    # Manejar el caso en que la Serie esté vacía
        max_hours_player= 'Player Not Found'
    df_genres.at[index, 'PlayerMH'] = max_hours_player


In [ ]:
## AÑO JUGADOR POR CADA JUGADOR

#primero debemos incluir el año en que jugó en una columna de users_items
df_users_items=df_users_items.merge(df_user_item_played_year, on=['item_id','user_id'],how='left')

# Itero sobre las filas de df_genresplayer
for index, row in df_genres.iterrows():
    genero = row['Genre']
    jugador = row['PlayerMH']
    
    # Filtro df_users_items para obtener las horas jugadas del jugador por género y año
    df_append_player = df_users_items[(df_users_items[genero] == 1) & (df_users_items['user_id'] == jugador)].groupby(by=['year'])['playtime_forever'].sum().reset_index()
    df_append_player = df_append_player.sort_values(by='playtime_forever', ascending=False)
    
    # Obtengo los años jugados y horas jugadas
    years_by_player = []
    for j in range(min(3, len(df_append_player))):
        this_year = '{Año: ' + str(int(df_append_player['year'].iloc[j])) + ', Horas: ' + str(int(df_append_player['playtime_forever'].iloc[j])) + '}'
        years_by_player.append(this_year)
    years_by_player=str(years_by_player)

    # Asigno la lista de años jugados y horas jugadas a la columna 'Years Played' en la fila actual
    df_genres.at[index, 'Years Played by player'] = years_by_player
    

In [ ]:
df_genres

In [ ]:
#se exporta el dataframe
df_genres.to_csv('Datos_procesados/genres_analysis.csv')

#### df_year_recommendations

In [ ]:
df_steamgames[['item_id','release_year']]

In [ ]:
'''
Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. 
(reviews.recommend = True y comentarios positivos/neutrales)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. 
(reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

'''



#### df_releaseyear_sentiment

In [ ]:
'''
Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios 
que se encuentren categorizados con un análisis de sentimiento.
# Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278} 
'''

# Load

### UsersRecommend

### UsersNotRecommend

In [ ]:
''' '''

def UsersNotRecommend( año : int ):

### sentiment_analysis

In [ ]:
''' '''

def sentiment_analysis( año : int ):

# EDA

# ML

In [ ]:
'''Si es un sistema de recomendación item-item:
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.'''

def recomendacion_juego( id de producto ):

In [ ]:
'''Si es un sistema de recomendación user-item:
 Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.
'''

def recomendacion_usuario( id de usuario ):